In [1]:
import requests
from bs4 import BeautifulSoup
import time
import re
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
def get_soup(url, verify=False):

    site = requests.get(url, verify=verify)
    
    return BeautifulSoup(site.content, 'html.parser')

In [3]:
url = "https://svo2.cab.inta-csic.es/theory/newov2/"
soup = get_soup(url)

In [4]:
def get_model_url(soup, model='BT-Settl'):

    model_entries = soup.find_all('td', class_='mdls2_modtit')

    # Search for the model
    model_href = None
    for entry in model_entries:
        b_tag = entry.find('b')
        if b_tag and model in b_tag.text:
            a_tag = b_tag.find('a')
            if a_tag and 'href' in a_tag.attrs:
                model_href = a_tag['href']
                break
    
    # Print the result
    if model_href:
        
        # Construct full URL if needed (if the href is relative)
        if not model_href.startswith('http'):
            full_url = url + model_href.split("newov2/")[1]
            return full_url
    else:
        print(f"{model} model not found in the page.")
        return

In [5]:
new_url = get_model_url(soup)
new_url

'https://svo2.cab.inta-csic.es/theory/newov2/index.php?models=bt-settl'

In [6]:
model_soup = get_soup(new_url)

In [10]:
def submit_params(url, model='BT-Settl', teff_min=5000, teff_max=5100, logg_min=4, logg_max=4.5, meta_min=0, meta_max=0.5):

    model = model.lower()

    # Prepare the form data with your desired parameters
    form_data = {
        'models': ','+model,
        'oby': '',
        'odesc': '',
        'sbut': '',
        # Set Teff min and max
        f'params[{model}][teff][min]': str(teff_min),
        f'params[{model}][teff][max]': str(teff_max),
        # Set logg min and max
        f'params[{model}][logg][min]': str(logg_min),
        f'params[{model}][logg][max]': str(logg_max),
        # Set metallicity min and max
        f'params[{model}][meta][min]': str(meta_min),
        f'params[{model}][meta][max]': str(meta_max),
        # Show all results
        'nres': 'all',
        # Submit button
        'boton': 'Search',
        # Selected model
        'reqmodels[]': model
    }

    # Submit the form without following redirects to see if there's a redirect
    response = requests.post(url, data=form_data, allow_redirects=False, verify=False)
    
    # If there's a redirect (status code 302), get the new location
    if response.status_code == 302:
        final_url = response.headers['Location']
        # Make sure the URL is absolute
        if not final_url.startswith('http'):
            final_url = f"https://svo.cab.inta-csic.es{final_url}"
    else:
        # If no redirect, the final URL is the original URL with possible query params
        final_url = response.url
    
    # Now follow the redirect if needed to get the actual content
    response = requests.post(url, data=form_data, allow_redirects=True, verify=False)

    if response.status_code == 200:

        print('Form submitted successfully!')
        return BeautifulSoup(response.text, 'html.parser')

    else:
        return

# Example usage
form_soup = submit_params(
    url = new_url,
    teff_min=5000, 
    teff_max=5100, 
    logg_min=4, 
    logg_max=4.5, 
    meta_min=0, 
    meta_max=0.5
)

Form submitted successfully!
